## Handwritten Digits Classification Using the MNIST Dataset

In [3]:
from torchvision import transforms, datasets
import torch
from torch import nn, optim
import torch.nn.functional as F

download and load training and testing data

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5, 0.5,), (0.5, 0.5, 0.5, 0.5,)),
])

train = datasets.MNIST('mnist_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(train, batch_size=64, shuffle=True)

test = datasets.MNIST('mnist_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(test)

create model

In [21]:
model = nn.Sequential(nn.Linear(784, 128),
                      nn.ReLU(),
                      nn.Dropout(p=0.2),
                      
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Dropout(p=0.2),
                      
                      nn.Linear(64, 10),
                      nn.LogSoftmax(dim=1))

train model

In [22]:
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.003)

epochs = 30
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        images = images.view(images.shape[0], -1) # flatten images into 784-long vectors
        
        optimizer.zero_grad()
        
        output = model.forward(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    print("Training loss (epoch %d):" % (e + 1), running_loss / len(trainloader))

Training loss (epoch 1): 1.9638300193652416
Training loss (epoch 2): 1.0885235772712398
Training loss (epoch 3): 0.7305113810466043
Training loss (epoch 4): 0.5962117362632426
Training loss (epoch 5): 0.5266355815917444
Training loss (epoch 6): 0.4778193776636744
Training loss (epoch 7): 0.44163732151232804
Training loss (epoch 8): 0.42186741202052974
Training loss (epoch 9): 0.3978890733900609
Training loss (epoch 10): 0.38211232832055103
Training loss (epoch 11): 0.36573426102969187
Training loss (epoch 12): 0.35326499274290446
Training loss (epoch 13): 0.34087350715134446
Training loss (epoch 14): 0.3318186255374443
Training loss (epoch 15): 0.32436797912441084
Training loss (epoch 16): 0.31146048214326283
Training loss (epoch 17): 0.3023461396776156
Training loss (epoch 18): 0.2932683113239594
Training loss (epoch 19): 0.28631357831169546
Training loss (epoch 20): 0.2785105089436589
Training loss (epoch 21): 0.2722223752152437
Training loss (epoch 22): 0.26456515028723265
Training 

test and validate

In [25]:
test_correct = 0
test_total = len(testloader)

with torch.no_grad():
    model.eval() # turn off dropout
    for image, label in testloader:
        image = image.view(1, 784)

        logits = model.forward(image)

        probabilities = F.softmax(logits, dim=1)
        probabilities_list = probabilities[0].numpy().tolist()
        max_probability = max(probabilities_list)
        prediction = probabilities_list.index(max_probability)

        if prediction == label:
            test_correct += 1
    model.train() # turn on dropout
    
print("Accuracy: ", test_correct / test_total)

Accuracy:  0.9502
